## Training a Wide ResNet with Dask Classifier

In [1]:
from dask.distributed import Client

In [2]:
import os
os.chdir('/home/ubuntu/adadamp-experiments')

import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import csv
from copy import copy
from adadamp.adadamp import DaskClassifier, DaskClassifierExpiriments

In [3]:
# training client
from dask.distributed import Client
from dask.distributed import performance_report

def _prep():
    from distributed.protocol import torch

client = Client(processes=False)
client.run(_prep)
client

Client Scheduler: inproc://172.31.40.124/30206/1 Dashboard: http://172.31.40.124:8787/status,Cluster Workers: 1 Cores: 4 Memory: 16.48 GB


In [4]:
from model import Wide_ResNet

client.upload_file("./exp-dask/model.py")

In [5]:
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

In [6]:
train_set = torchvision.datasets.CIFAR10(root='./exp-dask/data', train=True, download=True, transform=transform_train)
test_set = torchvision.datasets.CIFAR10(root='./exp-dask/data', train=False, download=True, transform=transform_test)
classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


In [7]:
# model for expiriment 1: 
#    “Decaying learning rate” follows the original implementation; 
#     the batch size is constant, while the learning rate repeatedly 
#     decays by a factor of 5 at a sequence of steps
# my understanding is that this is the "control" expeririment, where we are not touching the number of 
# workers nor the batch size
device = "cpu" if not torch.cuda.is_available() else "cuda:0"
model = DaskClassifierExpiriments(
    module=Wide_ResNet,
    module__depth=16,
    module__widen_factor=4,
    module__dropout_rate=0.3,
    module__num_classes=len(classes),
    loss=nn.CrossEntropyLoss,
    optimizer=torch.optim.SGD,
    optimizer__lr=0.1,
    optimizer__momentum=0.9,
    optimizer__nesterov=True,
    optimizer__weight_decay=0.5e-3,
    batch_size=128,
    max_epochs=200,
    device=device,
    grads_per_worker=128,
    client=client
)

In [8]:
# v0 - broken dist.py, LR 1
# v1 - fixed dist.py, LR 1
# v2 - LR 0.02
# v3 - LR 0.02, decaying learning rate, fixed data load

def write_stats(hist, epoch, exp, tag):
    toCSV = hist
    with open('./exp-dask/stats/{}/exp-{}-ep{}.csv'.format(exp, tag, epoch), 'w', encoding='utf8', newline='') as output_file:
        fc = csv.DictWriter(output_file, fieldnames=toCSV[0].keys())
        fc.writeheader()
        fc.writerows(toCSV)

In [9]:
def test(epoch, model, test_set):
    """
    tests if model meets certain testing standards:
    - by 10th epoch, accuracy is over 70%
    """
    if epoch == 10:
        print("[TEST] Testing accuracy for 10th epoch is over 70%")
        score = model.score(test_set)
        acc = model._meta['score__acc']
        if acc < 0.70:
            print("[TEST] Test failed with {} accuracy".format(acc))
            return False
        else:
            print("[TEST] Test passed with {} accuracy".format(acc))
    
    return True

In [10]:
def train(model, train_set, test_set, n_epochs=200, epoch_sched=[], lr_sched=[], bs_sched=[], log_interval=1, exp='increasing-bs'):
    """
    Train based on expiriment params
    
    Parameters:
    epoch_sched: update lr and bs at epochs in this list
    lr_sched: update lr to value at matching epoch. Should be same length as epoch_sched
    bs_sched: update bs to value at matching epoch. Should be same length as epoch_sched
    """
    assert len(epoch_sched) == len(lr_sched) == len(bs_sched), "Invalid schedules. Epoch, lr and bs schedules should all be the same length."
    
    epochs = copy(epoch_sched)
    lrs = copy(lr_sched)
    bss = copy(bs_sched)
    
    history = []
    for epoch in range(n_epochs):
        # check for updates
        if len(epochs) > 0 and epochs[0] == epoch:
            lr = lrs.pop(0)
            bs = bss.pop(0)
            epochs.pop(0)
            model.set_lr(lr)
            model.set_bs(bs)
            print("[UPDATE] Updated model params:\n\tlr: {}\n\tbs: {}".format(lr, bs))
            if epoch != 0:
                print("[UPDATE] Running loss on train set...", end=" ")
                train_score = model.score(train_set)
                print(model._meta['score__loss'], '(acc:', model._meta['score__acc'], ')')
        # run
        print("[Epoch {}]".format(epoch), end="")
        model.partial_fit(train_set)
        score = model.score(test_set)
        datum = {"epoch": epoch, "score": score, **model.get_params(), **model.meta_}
        print(" Score: {}".format(score))
        history.append(datum)
        
        # test
        if test(epoch, model, test_set) == False:
            print("[TEST] Test failed, exiting")
            break
        
        if epoch % log_interval == 0:
            write_stats(history, 'ep{}'.format(epoch), 'increasing-bs', 'v4')

    return history

In [11]:
# ie "Update LR to 0.1 and bs t0 640 on 60th epoch"
exp0_epochs = [0, 60, 120, 180]
exp0_lr = [
    0.02, 
    0.02 / 5, 
    0.02 / 5 / 5, 
    0.02 / 5 / 5 / 5
]
exp0_bs = [128, 128, 128, 128]
# train
hist = None
with performance_report(filename="dask-report.html"):
    hist = train(model, 
             train_set, 
             test_set, 
             n_epochs=200, 
             epoch_sched=exp0_epochs, 
             lr_sched=exp0_lr, 
             bs_sched=exp0_bs, 
             log_interval=20,
             exp='increasing-bs'
            )

[UPDATE] Updated model params:
	lr: 0.02
	bs: 128
[Epoch 0]

/home/ubuntu/adadamp-experiments/adadamp/adadamp/_dist.py:185: UserWarning: Model appears not to update with weight difference {diff}
  warn("Model appears not to update with weight difference {diff}")


 Score: 0.5777999758720398
[Epoch 1] Score: 0.6837999820709229
[Epoch 2] Score: 0.7258999943733215
[Epoch 3] Score: 0.7669000029563904
[Epoch 4] Score: 0.7827999591827393
[Epoch 5] Score: 0.8070999979972839
[Epoch 6] Score: 0.8023999929428101
[Epoch 7] Score: 0.8172999620437622
[Epoch 8] Score: 0.8348999619483948
[Epoch 9] Score: 0.8319000005722046
[Epoch 10] Score: 0.8432999849319458
[TEST] Testing accuracy for 10th epoch is over 70%
[TEST] Test passed with 0.8450999855995178 accuracy
[Epoch 11] Score: 0.8392999768257141
[Epoch 12] Score: 0.8456999659538269
[Epoch 13] Score: 0.8502999544143677
[Epoch 14] Score: 0.8495000004768372
[Epoch 15] Score: 0.852899968624115
[Epoch 16] Score: 0.8586999773979187
[Epoch 17] Score: 0.8610999584197998
[Epoch 18] Score: 0.8637999892234802
[Epoch 19] Score: 0.8703999519348145
[Epoch 20] Score: 0.8587999939918518
[Epoch 21] Score: 0.864799976348877
[Epoch 22] Score: 0.8640999794006348
[Epoch 23] Score: 0.8718000054359436
[Epoch 24] Score: 0.8687999844

In [12]:
write_stats(hist, '-final', 'increasing-bs', 'v4')

In [10]:
# ie "Update LR to 0.1 and bs t0 640 on 60th epoch"
exp1_epochs = [0, 60, 120, 180]
exp1_lr = [0.02, 0.02, 0.02, 0.02]
exp1_bs = [128, 640, 3200, 16000]
# train
hist = train(model, 
             train_set, 
             test_set, 
             n_epochs=200, 
             epoch_sched=exp1_epochs, 
             lr_sched=exp1_lr, 
             bs_sched=exp1_bs, 
             log_interval=20,
             exp='increasing-bs'
            )

In [ ]:
# ie "Update LR to 0.1 and bs t0 640 on 60th epoch"
exp2_epochs = [0, 60, 120, 180]
exp2_lr = [0.1, 
           0.1, 
           0.1 / 5, 
           0.1 / 5 / 5
          ]
exp2_bs = [128, 640, 640, 640]
# train
hist = train(model, train_set, test_set, n_epochs=200, epoch_sched=exp1_epochs, lr_sched=exp1_lr, bs_sched=exp1_bs)

In [ ]:
# ie "Update LR to 0.1 and bs t0 640 on 60th epoch"
exp3_epochs = [0, 60, 120, 180]
exp3_lr = [0.1, 
           0.1 / 5, 
           0.1 / 5 / 5,
           0.1 / 5 / 5 / 5
          ]
exp3_bs = [128, 128, 128, 128]
# train
hist = train(model, train_set, test_set, n_epochs=200, epoch_sched=exp1_epochs, lr_sched=exp1_lr, bs_sched=exp1_bs)

In [ ]:
toCSV = model.curr_metas
with open('./exp-dask/exp1-v0.csv', 'w', encoding='utf8', newline='') as output_file:
    fc = csv.DictWriter(output_file, fieldnames=toCSV[0].keys())
    fc.writeheader()
    fc.writerows(toCSV)